In [1]:
import scipy.sparse as sps
from Recommenders.Hybrid.ItemUserHybridKNNRecommender import ItemUserHybridKNNRecommender
from Utils.Evaluator import EvaluatorHoldout
import Utils.Reader as Reader
from Data_manager.split_functions.split_train_validation_random_holdout import\
    split_train_in_two_percentage_global_sample

In [ ]:
from Recommenders.GraphBased.RP3betaRecommender import RP3betaRecommender
#1
import numpy
import itertools
from tqdm.notebook import tqdm
# GRID SEARCH RP3BETA
best=0
config={
    "k":0,
    "a":0,
    "b":0

}
cutoff_list=[10]
from Recommenders.RP3betaCBFRecommender import RP3betaCBFRecommender
URM_all=Reader.read_train_csr(values_to_replace={0:0.2})

URM_train, URM_test = split_train_in_two_percentage_global_sample(URM_all, train_percentage = 0.8)
URM_train, URM_validation = split_train_in_two_percentage_global_sample(URM_train, train_percentage = 0.8)

evaluator_validation = EvaluatorHoldout(URM_validation, cutoff_list=cutoff_list)
evaluator_test = EvaluatorHoldout(URM_test, cutoff_list=cutoff_list)
rp3betaCBF_recommender = RP3betaRecommender(
    URM_train=URM_train,
    verbose=False
)

TopkList=numpy.arange(100, 800, 100)
aList=numpy.arange(0.1, 0.8, 0.1)
bList=numpy.arange(0.1, 0.8, 0.1)
implicit=[True,False]
for x in tqdm(itertools.product(TopkList,aList,bList,implicit),total=int(len(TopkList)*len(aList)*len(bList)*2), position=0, leave=True):
    k=x[0]
    a=x[1]
    b=x[2]
    i=x[3]
    rp3betaCBF_recommender.fit(
        topK=k,
        alpha=a,
        beta=b,
        implicit=i
    )

    result_df, _ = evaluator_validation.evaluateRecommender(rp3betaCBF_recommender)
    if result_df["MAP"][10] > best:
        best=result_df["MAP"][10]
        config={
            "k":k,
            "a":a,
            "b":b,
            "implici":i
        }
        #open text file
        text_file = open("result_experiments/grid_search/rp3beta.txt", "w")

        #write string to file
        text_file.write(str(config) +str(best))

        #close file
        text_file.close()

best,config

Unique UserID in the URM are 41629
Unique UserID in the URM and ICM are 41629
5826506
EvaluatorHoldout: Ignoring 759 ( 1.8%) Users that have less than 1 test interactions
EvaluatorHoldout: Ignoring 318 ( 0.8%) Users that have less than 1 test interactions


  0%|          | 0/686 [00:00<?, ?it/s]

/Users/fasa/PycharmProjects/RecSys-Challenge-2022-Polimi/Utils/metrics.py:276: RuntimeWarning: overflow encountered in power
  return np.sum(np.divide(np.power(2, scores) - 1, np.log2(np.arange(scores.shape[0], dtype=np.float64) + 2)),
/Users/fasa/PycharmProjects/RecSys-Challenge-2022-Polimi/Utils/metrics.py:270: RuntimeWarning: invalid value encountered in double_scalars
  ndcg_ = rank_dcg / ideal_dcg


EvaluatorHoldout: Processed 40870 (100.0%) in 32.72 sec. Users per second: 1249


In [ ]:
best,config